In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('GroupByJoin') \
    .getOrCreate()

25/02/28 10:22:14 WARN Utils: Your hostname, MichaelAngelo resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/02/28 10:22:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/28 10:22:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet("/mnt/c/Users/mab03/Desktop/DataEngineeringZoomcamp2025/TaxiData/green_data/*")

In [3]:
df_green.createOrReplaceTempView('green_taxidata')

In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green_taxidata
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue.show(5)

+-------------------+----+-------------+--------------+
|               hour|zone|       amount|number_records|
+-------------------+----+-------------+--------------+
|2062-08-15 00:00:00|   7|  3.600000000|             1|
|2020-01-01 00:00:00|  74|337.690000000|            26|
|2020-01-01 02:00:00| 260|235.170000000|            20|
|2020-01-01 09:00:00|  42| 79.520000000|             8|
|2020-01-01 13:00:00| 243| 42.550000000|             1|
+-------------------+----+-------------+--------------+
only showing top 5 rows



In [6]:
# Can then save results to parquet files and repartition to 20 files.
# df_green_revenue \
#     .repartition(20) \
#     .write.parquet('data/report/revenue/green', mode='overwrite')

In [8]:
import os
path = "/mnt/c/Users/mab03/Desktop/DataEngineeringZoomcamp2025/TaxiData/yellow_data/"
parquet_files = [f for f in os.listdir(path) if "_normalized" in f and f.endswith(".parquet")]

df_yellow = spark.read.parquet(*[os.path.join(path, f) for f in parquet_files])

In [9]:
df_yellow.createOrReplaceTempView('yellow_taxidata')

In [10]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow_taxidata
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [11]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [12]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [13]:
df_join.show(10)

+-------------------+----+-------------+--------------------+--------------+---------------------+
|               hour|zone| green_amount|green_number_records| yellow_amount|yellow_number_records|
+-------------------+----+-------------+--------------------+--------------+---------------------+
|2020-01-01 00:00:00| 122|103.960000000|                   2|          NULL|                 NULL|
|2020-01-01 00:00:00| 188|127.570000000|                   6| 105.720000000|                    5|
|2020-01-01 00:00:00| 226| 41.150000000|                   3| 233.210000000|                   13|
|2020-01-01 00:00:00| 234|         NULL|                NULL|8836.020000000|                  481|
|2020-01-01 00:00:00|   3|         NULL|                NULL|  25.000000000|                    1|
|2020-01-01 00:00:00|  32| 68.950000000|                   2|  18.000000000|                    1|
|2020-01-01 00:00:00|  59| 50.900000000|                   3|          NULL|                 NULL|
|2020-01-0

In [14]:
# Again can save it if needed
# df_join = spark.read.parquet('data/report/revenue/total')